Going to focus on the 2024 pop but should be widely applicable beyond

In [5]:
import boto3
import pandas as pd
import os

In [3]:
# Specify your S3 bucket and CSV file paths
bucket_name = 'tfl-cycle-data'
csv_dir = 'JourneyInfo/'

CSVs = ['387JourneyDataExtract01Jan2024-14Jan2024.csv',
        '388JourneyDataExtract15Jan2024-31Jan2024.csv',
        '389JourneyDataExtract01Feb2024-14Feb2024.csv',
        '390JourneyDataExtract15Feb2024-29Feb2024.csv',
        '391JourneyDataExtract01Mar2024-14Mar2024.csv',
        '392JourneyDataExtract15Mar2024-31Mar2024.csv',
        '393JourneyDataExtract01Apr2024-14Apr2024.csv',
        '394JourneyDataExtract15Apr2024-30Apr2024.csv']


# Initialize a session using Amazon S3
s3 = boto3.client('s3')
for csv in CSVs:
    # Download the CSV file from S3
    s3.download_file(bucket_name, f"{csv_dir}{csv}", f"temp/{csv}")

In [6]:
# Path to the directory containing CSV files
directory_path = 'temp/'

# List to hold dataframes
dataframes = []

# Read all CSV files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)

# Merge all dataframes into a single dataframe
merged_dataframe = pd.concat(dataframes, ignore_index=True)

In [7]:
merged_dataframe.head()

,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
0,138583046,2024-04-14 23:58,1190,"Kennington Lane Rail Bridge, Vauxhall",2024-04-15 00:01,1073,"Kennington Road Post Office, Oval",50789,CLASSIC,3m 27s,207988
1,138583048,2024-04-14 23:58,993,"Drummond Street , Euston",2024-04-15 00:16,1227,"Devonshire Terrace, Bayswater",55839,CLASSIC,17m 52s,1072179
2,138583049,2024-04-14 23:58,1090,"Warren Street Station, Euston",2024-04-15 00:05,978,"Russell Square Station, Bloomsbury",60337,PBSC_EBIKE,6m 32s,392736
3,138583050,2024-04-14 23:58,200235,"Manbre Road, Hammersmith",2024-04-15 00:12,1016,"Hereford Road, Bayswater",60193,PBSC_EBIKE,13m 37s,817269
4,138583042,2024-04-14 23:57,200175,"Wandsworth Rd, Isley Court, Wandsworth Road",2024-04-15 00:05,300006,"Clapham South, Clapham South",60358,PBSC_EBIKE,8m 17s,497745


In [9]:
merged_dataframe.tail()

,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
2454642,136450337,2024-01-01 00:03,1034,"Bruton Street, Mayfair",2024-01-01 01:29,3452,"Panton Street, West End",60538,PBSC_EBIKE,1h 26m 2s,5162949
2454643,136450332,2024-01-01 00:01,300235,"York Way, Kings Cross",2024-01-01 00:31,200212,"Blackfriars Station, St. Paul's",54136,CLASSIC,29m 54s,1794054
2454644,136450333,2024-01-01 00:01,300235,"York Way, Kings Cross",2024-01-01 00:31,200212,"Blackfriars Station, St. Paul's",57461,CLASSIC,29m 43s,1783017
2454645,136450334,2024-01-01 00:01,300235,"York Way, Kings Cross",2024-01-01 00:30,200212,"Blackfriars Station, St. Paul's",21426,CLASSIC,29m 23s,1763701
2454646,136450335,2024-01-01 00:02,200132,"Gaywood Street, Elephant & Castle",2024-01-01 00:04,1076,"Walnut Tree Walk, Vauxhall",56710,CLASSIC,2m 56s,176167


In [26]:
# get unique station number name lookup
docks = merged_dataframe[['Start station number', 'Start station']].drop_duplicates()
docks.columns = ['stationID', 'StationName']
docks = docks.sort_values('stationID').reset_index(drop=True)
print(docks)

     stationID                        StationName
0          959            Milroy Walk, South Bank
1          960          Hop Exchange, The Borough
2          961          Union Street, The Borough
3          962        Stamford Street, South Bank
4          963             Bankside Mix, Bankside
..         ...                                ...
798     300248        Sopwith Way, Battersea Park
799     300249      Westminster Pier, Westminster
800     300250             Temple Gardens, Temple
801     300252  Canada Water Station, Rotherhithe
802     300253     Bermondsey Station, Bermondsey

[803 rows x 2 columns]


In [15]:
# Dataframes representing inflows and outflows

docks_2024_out = merged_dataframe[['Start date', 'Start station number', 'Start station']]
docks_2024_out.columns = ['date_time', 'stationID', 'stationName']
docks_2024_in = merged_dataframe[['End date', 'End station number', 'End station']]
docks_2024_in.columns = ['date_time', 'stationID', 'stationName']

In [20]:
df = docks_2024_in.groupby('stationName').agg('count')
print(df.sort_values('stationID'))

                                               date_time  stationID
stationName                                                        
Eaton Square, Belgravia                               10         10
Mechanical Workshop Penton                            16         16
Mechanical Workshop Clapham                          131        131
Here East South, Queen Elizabeth Olympic Park        283        283
Colombo Street, Southwark                            302        302
...                                                  ...        ...
Wormwood Street, Liverpool Street                  11617      11617
Hop Exchange, The Borough                          12183      12183
Brushfield Street, Liverpool Street                12376      12376
Hyde Park Corner, Hyde Park                        13468      13468
Waterloo Station 3, Waterloo                       17914      17914

[805 rows x 2 columns]
